In [6]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import os.path
import json
import pickle
import requests
import asyncio
from tenacity import retry, wait_exponential, stop_after_attempt, retry_if_exception_type


def oauth():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 
              'https://www.googleapis.com/auth/script.projects', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file'
              ]
    creds_filename = 'test-daebong-service-account.json'  # 서비스 계정 파일 경로를 지정합니다.

    # 서비스 계정 파일을 사용하여 인증 정보를 로드합니다.
    creds = service_account.Credentials.from_service_account_file(creds_filename, scopes=SCOPES)
    return creds

def oauthByWeb():
    # 필요한 스코프를 지정합니다.
    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/script.projects',
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.file'
    ]
    creds = None
    # 'token.json' 파일이 존재하면, 저장된 인증 정보를 불러옵니다.
    if os.path.exists('token.json'):
        with open('token.json', 'rb') as token:
            creds = pickle.load(token)

    # 저장된 인증 정보가 없거나, 유효하지 않은 경우 새로운 인증을 진행합니다.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            creds_filename = 'oauth-new-daebong.json'
            flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
            creds = flow.run_local_server(port=8080)
        # 새로운 인증 정보를 'token.json'에 저장합니다.
        with open('token.json', 'wb') as token:
            pickle.dump(creds, token)

    return creds



In [7]:
def col_to_index(col):
    index = 0
    for c in col:
        index = index * 26 + (ord(c) - ord('A') + 1)
    return index - 1

def convert_range_to_indices(sheet_id, cell_range):
    """
    주어진 셀 범위를 rowIndex와 columnIndex로 변환합니다.
    
    :param sheet_id: 변환할 시트의 ID
    :param cell_range: 변환할 셀 범위 (예: "A2:B")
    :return: 변환된 범위 딕셔너리
    """
    import re
    
    # A1 표기법을 정규 표현식으로 분리
    match = re.match(r"([A-Z]+)([0-9]+):([A-Z]+)([0-9]+)", cell_range)
    if not match:
        raise ValueError("셀 범위 형식이 올바르지 않습니다. 예: 'A2:B10'")
    
    start_col, start_row, end_col, end_row = match.groups()
    
    start_row_index = int(start_row) - 1
    end_row_index = int(end_row)
    start_col_index = col_to_index(start_col)
    end_col_index = col_to_index(end_col) + 1
    
    return {
        'sheetId': sheet_id,
        'startRowIndex': start_row_index,
        'endRowIndex':end_row_index,
        'startColumnIndex': start_col_index,
        'endColumnIndex': end_col_index
    }

In [5]:
@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
async def batchWriteRequest(service, spreadsheet_id, all_requests):
    body = {
        'requests': all_requests
    }

    # batchUpdate 호출
    response = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body=body
    ).execute()
    
    return response


In [12]:

@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
async def delete_sheet_by_title(service_sheets, destination_spreadsheet_id):
    spreadsheet = service_sheets.spreadsheets().get(spreadsheetId=destination_spreadsheet_id).execute()
    sheets = spreadsheet.get('sheets', '')
    sheet_order_id_발주서 = None
    sheet_order_id_누적발주 = None
    sheet_order_id_입금내역 = None
    sheet_order_id_상품목록 = None
    
    for sheet in sheets:
        if sheet.get('properties', {}).get('title', '') == "발주서이전버전":
            sheet_order_id_발주서 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "상품목록이전버전":
            sheet_order_id_상품목록 = sheet.get('properties', {}).get('sheetId', '')


    print(sheet_order_id_발주서)
    print(sheet_order_id_상품목록)
    requests = []
    
    if sheet_order_id_상품목록:
        requests.append({
            "deleteSheet": {
                "sheetId": sheet_order_id_상품목록
            }
        })

    # if sheet_order_id_입금내역:
    #     requests.append({
    #         "deleteSheet": {
    #             "sheetId": sheet_order_id_입금내역
    #         }
    #     })
    
    if sheet_order_id_발주서:
        requests.append({
            "deleteSheet": {
                "sheetId": sheet_order_id_발주서
            }
        })
    # if sheet_order_id_누적발주:
    #     requests.append({
    #         "deleteSheet": {
    #             "sheetId": sheet_order_id_누적발주
    #         }
    #     })

    delete_request_body = {
        "requests": requests
    }

    if len(requests)>0:
        service_sheets.spreadsheets().batchUpdate(
            spreadsheetId=destination_spreadsheet_id,
            body=delete_request_body
        ).execute()


In [16]:
filtered_user_data = [
        # ['20', '1D86pGcpOsFWp6IWb_KCGnNJerYcIiFNYzptaR2m21WI', '권오상(오에스인'], ['21', '1U_HI5Th-1Yh9RBWiOGljs-NhXazxPl1HkbPQ9IZrKzA', '왕선균(오늘곳간)'], ['42', '1ziAp-yyjSgHesi8OKrzl9YxQYufx8Q9rUhUFaT5pWzI', '42'], ['47', '1WgI0QIOdj_NS7JdtIgDdYr-EAI5mTVnjQV3iVm5PP40', '47'], ['48', '1JScmTyIC8YKuRE867hRjiP2_eh4DAVmlJMw-zdhKw2s', '48랑랑이네'], ['50', '1MOpKL9N6ZhdqR6OBEq7AL2YekDCZeLs1VOJv35VphEw', '장진웅'], ['56', '1XmTgyrxTdK3hcHUm1YwYkoBom51v-y2jZTwC0A1Bn8U', '송용재'], ['57', '1ps2R_HXXHEuoKz7glv6LzlBZVyMNvfYNx23kQOxdnXE', '57'], ['60', '1erbDgg1807ZRZhRsoZJr6uYlGgLpT557dt81JxsYO-w', '60'], ['90', '1ZTgyd5YQjluAL8pLVKVqk-BqrOl6EgST47Gd8OtP_RA', '90'], ['92', '15DEt2XorXfKa-UlJtHsauu2b4dmS58RJ3fx1fS1zwkY', '92'], ['97', '1andpTzI55GWGRXV-JjaAtSIE3m4i7fDCHU2CNSk9TEU', '방성식'], ['98', '1ppOTzTz8_QMET4m2z0i6-aMg136hwfi1VeL2HCACyh4', '강대혁'], ['114', '1mZVlL4QwI2eHByQEpYs0dLYUQX4OcSa4tsdTJPR2VRI', '114'], ['121', '1ZEtoHfB5c0PJe2cE7TnP5Q5j8nLrc8hF7lPpdemH90A', '121'], ['130', '1MZN4QGtYktUqUXd6DFPxfyIustQ2BciBQdZd7-8jiOU', '보연주식회사'], ['138', '1Ync5OXAnvBk847u0dDiNkg-H11Ma3CKWzu036NCvPUg', '138'], ['143', '1LbXFDggoMRGJPUTgYp9I5BRvJZ3Fm20oQVL82ymRUl0', '143'], ['171', '1ZOX9E8BaYZ9IvVYZ1AeIQ7--PW_QYv7uHviHP-bqOyU', '우리과일맛있게맛'], ['178', '1hoPzaAfLCA3sV_WMrgNd_aQlIXEkBty4nwVcDZG1Pq0', '178무지개식탁'], ['180', '1WkE6NCdspuCQVIi2mGBXiA1GaFN05-l_9y9Iymu3Ln0', '180'], ['189', '1O1F3u9rP1w1zPCBwvgBhzOZxZdcHxgh3ZqbsRBBWI14', '안호현(마싯농)'], ['192', '1TSnCCP6nJ4PljZBsoqru23j8d53pfLJf5S0pK6tDAXM', '192_하이푸드'], ['195', '1q_g0-xOyXOm02BLASm7kyasalv6bWGO2RBF-Gbn6gGw', '다오소'], ['208', '1djE04qMRnEYYLwY0y4qEpHnBVWwOBxAimrzLgTuBe6k', '２０８'], ['209', '1GBPzy-GuxujWEGkH606Eb72-D8lo4yEKrZP7DrGzWFk', '209'], ['226', '1-J6V0nLHQlxLA8PbTiTXaGt08VlqGv5fo3kfaH35FJE', '김정재'], ['230', '1OxaNX2XUK5CTsqseH6Dnj_3Bhi1FNn_79YAAH1sOdhE', '230'], ['260', '1xD9hJnw7s8BNMuFGpC7uiO2YJsjNPRYQoObvNsxo4sE', '송현우'], ['276', '1V0EBc39eBX1ekf8DGFvCiS5SuVnTm6TehpOkhof7Jlo', '276_김도환(청년'], ['283', '1ldQveZcPwtieVcN-zlIzhXBxO3xnY7523gNNQpnPrfE', '283'], ['325', '14SbtLIs4ZPfiaJh-kRb-TE4RtOcO-gw4knuy3MolLX8', '이호연(컨프래'], ['328', '1wcx04SVtzkmlPl-PvqihBnRYHXnaNZ6EJykoHjME5hQ', '전혜린'], ['330', '1bwyuRqmCmNMr0-0qv_kglHL2olWFx-92MvbF-L8iRZY', '330'], ['335', '13bZWoE-oHVuPaPQXRYxIjDDAqmpvnqETYuP5S94Re08', ''], ['337', '10JQGUWjXCWbhmL7fq8cJfCmtBBIiDaOrqdjh1U1EI_U', '337_박정윤'], ['339', '1GCUBnyUr25uhB3HozLbiUQ5JngLhr7rfoNNR1cxAo58', '339슈거팜'], ['343', '1pd06-Meiiz1sntlIlMd49T6jS9Y6ZiXJU1gRs_hr_bE', '343'], ['353', '1jdRXB72CFDd35tj3Vmr7Ig2OXJfZX3x6wTLl0imas2M', '프리미엄찌니샵'], ['367', '1PXjuzLoiXYK9hLjmiSpsKMxnGXc09N_ZWxoaXPvMe-M', '367'], ['387', '1rNW5Kwv4M59HwihbR5FxiHq8MsvepJQsa1gRzrkmwME', '387'], ['406', '1fBNPp2YdC_xpb3W_4jGkZWRfV7l6_Vn96Vt44LQy8HE', '신윤식'], ['436', '1Ssu9s18Gug3HLRZNp3Hr_TddM2IOcPhlGE7ZjLRPsGo', '김홍곤(주윤)'], ['438', '1zWRhxaoNt8egvqUGmkRRf6KEFccJTDH-HleiKce9ZIM', '김윤성(오손도손)'], ['442', '1E2i1IJRuPAoDmXum5vjrNo65mmaEjIyBKhENEH7VZEE', '임경섭(프루딧)'], ['443', '1IOCubZrxV-dR1uNCFSgP3juhLIlMIMHYVaJACAMu1tY', '서영희(담다컴퍼'], ['461', '104ZWc3RoB0WjkITYQLyFBLvkZ5UywvIJZHoWIha9iTk', '우리동네남매마켓'], ['464', '1JG7u3zTmaH_WfKri2MvNGfUwGua3WAaCj-fXW6vSW2w', '조재훈(퍼플로드)'], ['468', '1dMLFlUP-oXYHnmY-Qm4hK0QUykH6sZyPdPza35HPWwU', '오정애'], ['470', '1oQvBGq3yKYDN9GxaK95bmVO99aVwDn9KGxJ_ceI2BFo', '최주연'], ['474', '1XRUFTiPVeZqLj9p1bw3t-qL-B-jy4Qteah6F1W216MA', '이우람'], ['475', '1ElTPyHmukcSKHDc4snsCcGfciEvOnngzXMB-3ccqdFo', '팔도장터'], ['485', '1vVGjqAgaOFovhHhrcpGkHmGktJug0f0zKgYIERT2FTA', '485정준희'], ['486', '1Uxgzhqwf-PLUE7OO9_Vzpkrr9i1N8PrVmvSpCjy6Pyo', '어썸파머스(주)'], ['489', '1RshPDxepseX4rzT6NxqCjVCDRu-I7zr8qUBgX3AxNQY', '이종철(수수마켓)'], ['496', '1xAcpRH5z51YF3dezUkBwc78vxWPfJOXquCN_4OkcuDM', '주식회사엠디어스'], ['497', '1bIOa6_1v7Jk3t2wAaPyo2G-hACN2iadRuNuVJbYMUVA', '497'], ['498', '1Bzna425shVhelODCpen2FUdiGu24KtR7jQUPNhaZkDY', '배민우'], ['500', '1asRBkSVowjMXIFWkeYZuqyTQmbpUhZi6ht3y66k4HE0', '함영민(과일파는'], ['503', '1np5z7X2XVUyTu1sOH_J0RREVPDNVK8CuN7CG8Jjf1K4', '주식회사와이앤제'], ['504', '1YwT7BaFpOKB2Ng3MViX7UGuF9hZJUJRBFJN3eXn4gHw', '504_팜브릿지'], ['506', '1HgYKDeGrRGxwg-j-yVqeZe6sJY1ip5mDJQ2lj4jLPt0', 'SH프레시푸드'], ['513', '1_eWdxS4Xdykn-GbZTTyOdH8f41GEvwURRcisG-PI9XU', '황소영'], ['514', '1Z0BJDjKsvPOrYXKD93NyRLUu--ecxpNue05n_UPLJjc', '주식회사호신호'], ['516', '1VhHatV-asswfTDa9M13B8Orf2aDSmbdYWBzwrI4dqjM', '김성헌'], ['517', '133IbYEp-aXQH-ECojwEEak6LKRgPKraS0yqTSvqNvWM', '김가람(과람아저'], ['523', '1y6sJA3EAAqaSLPgZMWYJjY5t2IZLrZGMBTXWI7sroIg', '김지영(동지코퍼'], ['525', '1zyM4XOmz1hRUwLM-2WXHh5BaJ-TMowTA0abn8cd7fe4', '525'], ['527', '1RZUWJpKa9b3CiBFNGkgUwHV2nJo1BE1gs9Uoe1XZmuM', '정민지'], ['537', '1kCgT5DWV1CcHTLJldd6BgiNU-Ara1zkX1TFKDMXiSms', '바이티존'], ['538', '1c3mQbrzmSfS19tAA7MWtVyP_DoT47KI3MPKhI24TUhE', '김규리,김경률'], ['541', '17sd8bF7sHRXpj8cEfTJ_44zhn8ALvSqJ26hkZlpU5ik', '그록정산금'], ['542', '1sClq5Zx710IIh5GhHzm6SdIgeZGmHVbmb8rNetSytJM', '조성현'], ['543', '1fmpcNUCK3wxx1M5TvCt1Cr00wtAaFwoUwPIFMnir5xw', '543_최혜정'], ['545', '1iC421AvucDQacamyihRfIP5WymejCEOC6Fd8t2vkre0', '최정훈(조니마켓)'], ['546', '1xUh--GSPTPLmMXXyHLiJ6DjyL3J9BG3py3SeSGJHkcw', '김혜연(어랑,과일'], ['547', '17A2-P3z_NA1BEGUXxRPEXHmpzRgKCmeA3hDuUWyQ3Xk', '이철호'], ['552', '1zt-VXJNilmSDYDzEhZqyV12Ern3WVu2-IivHWSJ4H1o', '가순성'], 
        ['553', '1SCC7A90vXM5srUZpubCyc02Px3RsCpOeUjC8_Nl5g88', '김지인'], ['554', '1F1N6d2Co-E7LbNfnuj_xfTcJczBdUcVjQugp8sUi_3o', '싱그린청과'], ['555', '1Pu9P4zFoDdFXK9ER_rZaEj2fa0FX36YEyCZIAV-YkEw', '(주)마니쿡스'], ['559', '1PLYZbwJy_4EqiWoQw7lMAC6RGc9f6-BEN92yZmAWRZI', '박지환(와치맨컴'], ['562', '1jYeZFz3CNgXkBV3z1CsoD8k_e6xH-vZa3M0KboBD8rk', '우리DC마켓']
        
]

In [20]:
def hide_columns(sheet_order_id):
    request = {
            "updateDimensionProperties": {
                "range": {
                    "sheetId": sheet_order_id,
                    "dimension": "COLUMNS",
                    "startIndex": 0,  # A열 (0-based index)
                    "endIndex": 7    # G열 (0-based index, exclusive)
                },
                "properties": {
                    "hiddenByUser": True
                },
                "fields": "hiddenByUser"
            }
        }

    return request

In [18]:
# #삭제하기 발주서이전버전 상품목록이전버전

# if __name__ == "__main__":
#     creds = oauthByWeb()
#     service_sheets = build('sheets', 'v4', credentials=creds)
#     for data in filtered_user_data:
#         spreadsheet_title = data[0]
#         requests = []
#         if 19 > int(spreadsheet_title):
#             continue

        


#         destination_spreadsheet_id = data[1]
#         print("spreadsheet_title", spreadsheet_title)

#         await delete_sheet_by_title(service_sheets, destination_spreadsheet_id)
#         print("result_발주서이전버전_상품목록이전버전_삭제")

